<a href="https://colab.research.google.com/github/kasra-keshavarz/varstool/blob/master/tests/notebook/plhs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import numpy as np


def plhs(sp, params, slices, seed=None, iter=50, criterion='maximin'):

    return plhs_sample


def slhs(sp, params, slices, seed=None, iter=100, criterion='maximin'):
    '''
    Description:
    ------------
    This function created SLHS samples, based on [1 and 2]


    Arguments:
    ----------
    :param lb: lower bound of the sequence
    :type lb: one of int, np.int32, np.int64
    :param ub: upper bound of the sequence
    :type ub: one of int, np.int32, np.int64
    :param slices: the number of slices
    :type slices: one of int, np.int32, np.int64
    :param seed: seed number for randomization
    :type seed: int, np.int32, np.int64
    :param iter: maximum iteration number 
    :type iter: int, np.int32, np.int64, optional
    :param criterion: the criterion for assessing the quality of sample points
                      the available options are: 'maximin' and 'correlation',
                      defaults to 'maximin'
    :type criterion: str, optional


    Returns:
    --------
    :return slhs_sample_x: the final slhs sample array based on 'x' criterion
    :rtype slhs_sample_x: np.array


    References:
    -----------
    .. [1] Ba, S., Myers, W.R., Brenneman, W.A., 2015. Optimal sliced Latin
           hypercube designs. Technometrics 57 (4), 479e487.
           http://dx.doi.org/10.1080/00401706.2014.957867
    .. [2] Sheikholeslami, R., & Razavi, S. (2017). Progressive Latin Hypercube 
           Sampling: An efficient approach for robust sampling-based analysis of 
           environmental models. Environmental modelling & software, 93, 109-126

    '''

    # define the seed number
    if seed:
        np.random.seed(seed)

    # Check the inputs and raise appropriate exceptions
    if type(criterion) is not str:
        raise TypeError(str(criterion)+" is not defined. Use either 'maximin'"+\
                                         " or 'correlation'.")

    # Check the criterion
    if criterion is 'maximin':
        sp = _sampler(sp, params, slices)

        return slhs_sample_maximin

    elif criterion is 'correlation':

        return slhs_sample_correl

    else:
        raise RuntimeError("Cannot generate the sample. Try again.")


def _sampler(sp, params, slices, seed=None) -> np.array:
    '''
    Description:
    ------------
    A simple sampling algorithm to create lhs slices.


    Arguments:
    ----------
    :param lb: lower bound of the sequence
    :type lb: one of int, np.int32, np.int64
    :param ub: upper bound of the sequence
    :type ub: one of int, np.int32, np.int64
    :param slices: the number of slices
    :type slices: one of int, np.int32, np.int64
    :param seed: seed number for randomization
    :type seed: int, np.int32, np.int64


    Returns:
    --------
    :return sample_array: the final sample array
    :rtype sample_array: np.array


    References:
    -----------
    .. [1] Ba, S., Myers, W.R., Brenneman, W.A., 2015. Optimal sliced Latin
           hypercube designs. Technometrics 57 (4), 479e487.
           http://dx.doi.org/10.1080/00401706.2014.957867.
    .. [2] Sheikholeslami, R., & Razavi, S. (2017). Progressive Latin Hypercube
           Sampling: An efficient approach for robust sampling-based analysis of
           environmental models. Environmental modelling & software, 93, 109-126
    '''
    # define the randomization seed number
    if seed:
        np.random.seed(seed)
    

    # check the dtype of input arguments
    msg = ("dtype of '{}' array must be 'int', 'np.int32' or 'np.int64'.")
    if type(sp) not in [int, np.int32, np.int64]:
        raise ValueError(msg.format('sp'))
    if type(params) not in [int, np.int32, np.int64]:
        raise ValueError(msg.format('sp'))
    if type(slices) not in [int, np.int32, np.int64]:
        raise ValueError(msg.format('sp'))

    # check the number of slices and sample points
    if (sp % slices) != 0:
        raise ValueError("sample points must be a multiplier of slices.")

    # check the sign of the input arguments
    sign_msg = ("the sign of '{}' must be positive (>0).")
    if sp < 0:
        raise ValueError(sign_msg.format('sp'))
    if params < 0:
        raise ValueError(sign_msg.format('params'))
    if slices < 0:
        raise ValueError(sign_msg.format('slices'))


    # calculate the number of slices
    slice_sp = sp // slices # to get int

    # generate slices using sampling (int) without permutation
    rand_perm = lambda slice_sp, slices: np.concatenate([np.random.permutation(slice_sp)+1 for _j in range(slices)])
    sample_array = np.stack([rand_perm(slice_sp, slices) for _i in range(params)])
    
    # DEBUG
    # print('sample_array:')
    # print(sample_array)
    # END DEBUG

    # positional function definition
    slice_spec = lambda row, slice_sp: np.stack([(row==_j+1) for _j in range(slice_sp)])

    # row-wise assessment
    for _row in range(0, sample_array.shape[0]):
        position_array = slice_spec(sample_array[_row, :], slice_sp)
        for kk in range(0, slice_sp):
            lb = (kk*slices)+1
            ub = (kk+1)*slices
            perm = _perm_intv(lb, ub, slices, seed)
            try:
                sample_array[_row, position_array[kk, :]] = perm
            except: # sometimes a number might be missing due to randomness...
                raise RuntimeError("error! Change seed number and try again.")
    sample_array = np.random.uniform(sample_array-1, sample_array)
    sample_array /= sp

    return sample_array.T


def _perm_intv(lb, ub, slices, seed=None) -> np.array:
    '''
    Description:
    ------------
    A simple random sampling given the lower and upper bounds,
    without permutation, and amongst the integers in the interval


    Arguments:
    ----------
    :param lb: lower bound of the sequence
    :type lb: one of int, np.int32, np.int64
    :param ub: upper bound of the sequence
    :type ub: one of int, np.int32, np.int64
    :param slices: the number of slices
    :type slices: one of int, np.int32, np.int64


    Returns:
    --------
    :return perm: the sampled np.array
    :type perm: np.array
    '''
    # define the randomization seed number
    if seed:
        np.random.seed(seed)

    # a simple sampling without permutation algorithm
    length = np.abs(ub-lb)+1
    perm   = np.arange(start=lb, stop=ub+1, step=1)
    for k in range(2, length+1):
        index1 = np.int(np.ceil(np.random.rand() * k))
        index2 = perm[k-1]
        perm[k-1] = perm[index1-1]
        perm[index1-1] = index2
    perm = perm[0:slices+1]
    
    # DEBUG
    # print('perm is:')
    # print(perm)
    # END DEBUG

    return perm

